<a href="https://colab.research.google.com/github/feliciakiani/PolitikPedia/blob/main/Machine%20Learning/ETL_Spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PySastrawi
!pip install flask
!pip install mysql.connector
!pip install textblob

In [32]:
import numpy as np
import pandas as pd
# from google.colab import files
# uploaded = files.upload()

import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import mysql.connector
from textblob import TextBlob
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import os

from keras.models import load_model

In [33]:
def filters(text):
  filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
  regex_pattern = '[' + re.escape(filters) + ']'

  filtered_text = re.sub(regex_pattern, '', text)

  return filtered_text


factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
    return stemmer.stem(text)

def preprocessedText(text):
  text = filters(text)
  text = stemming(text)

  return text

In [34]:
# #gk perlu
# with open('sentences_spam.txt', 'w', encoding='utf-8') as file:
#     for sentences_spam in X:
#         file.write(sentences_spam + '\n')

# from google.colab import files

# # Save the best model checkpoint to a file
# files.download('sentences_spam.txt')

In [35]:
model = load_model("spamDetection.h5")

def spam_detection(user_id):
  # Koneksi ke database Cloud SQL
  conn = mysql.connector.connect(
          host=os.getenv("DB_HOST"),
          user=os.getenv("DB_USERNAME"),
          password=os.getenv("DB_PASSWORD"),
          database=os.getenv("DB_NAME")
  )

  # Buat kursor
  cursor = conn.cursor()

  #-------------------------------------------------spam_Classification-------------------------------------------#
  # Hyperparameter, jangan diganti
  max_words = 100

  query_comments = f"SELECT ID as comment_id, Komentar as komentar_text FROM komentar WHERE IDUser = {user_id} ORDER BY TglKomentar DESC LIMIT 1"
  cursor.execute(query_comments)
  comments = cursor.fetchall()

  if comments:
    for row in comments:
      comment_id, komentar_text = row  # Extracting comment_id and Komentar
      print(f"Comment ID: {comment_id}, Komentar: {komentar_text}")

   # Apply preprocessing
    preprocessed_text = preprocessedText(komentar_text)

    # # Tokenize the sentences
    # tokenizer = tf.keras.preprocessing.text.Tokenizer()
    # # Read the content of 'sentences.txt' and split it into lines
    # with open('sentences.txt', 'r', encoding='utf-8') as file:
    #     tokenizer_sentences = file.read().splitlines()
    # tokenizer.fit_on_texts(tokenizer_sentences)

    # Tokenize the text data
    Tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer = Tokenizer(num_words=max_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True)
    # Read the content of 'sentences.txt' and split it into lines

    with open('sentences_spam.txt', 'r', encoding='utf-8') as file:
      tokenizer_sentences = file.read().splitlines()
    tokenizer.fit_on_texts(tokenizer_sentences)

    # Tokenize and pad the new text
    new_sequence = tokenizer.texts_to_sequences([preprocessed_text])
    new_padded_sequence = pad_sequences(new_sequence, maxlen=max_words, padding='post')

    # Make predictions
    prediction = model.predict(new_padded_sequence)

    # Extract the scalar value from the NumPy array
    confidence = prediction[0, 0]

    # Threshold for considering a label as positive
    threshold = 0.5

    # Interpret prediction
    predicted_class = 1 if confidence >= threshold else 0

    # Convert confidence to a format that can be handled by format method
    confidence_str = '{:.4f}'.format(confidence)

    print(f"User ID: {user_id}, Comment ID: {comment_id}, Predicted Class: {predicted_class} (Confidence: {confidence_str})")

    # Return the results
    result = {
        "comment_id": comment_id,
        "predicted_class": predicted_class,
        "confidence": confidence_str
    }
  else:
    result = {"error": f"Comment with ID {user_id} not found."}
  
  # Close the database connection
  cursor.close()
  conn.close()

  return result

In [36]:
# flask
from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route('/spam_detection/<string:userId>', methods=['GET'])
def spam_detection_endpoint(userId):

    result = spam_detection(userId)
    print(result)
    return jsonify(result)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8888)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8888
 * Running on http://192.168.1.56:8888
Press CTRL+C to quit


Comment ID: 36, Komentar: Selamat! 200 IMPoin dri isi ulang pulsa BERHASIL msk ke akunmu. Poin hangus pd 31-12-2023. Terus kumpulkan Poin & tukar dgn reward menarik di *999# atau bit.ly/imnm3


[2023-12-19 18:18:54,598] ERROR in app: Exception on /spam_detection/14 [GET]
Traceback (most recent call last):
  File "c:\Users\Felicia\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Felicia\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 869, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Felicia\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 867, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Felicia\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 852, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\Felicia\AppData\Local\Temp\ipykernel_28044\2098730259.py", line 9, in spam_detection_endpoint
    result = spam_detection(userId)
  File "C:\Users\Felicia\AppData